In [19]:
import os
import numpy as np
import random
import string
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


from dataset import *
from util import *
from model import *

import matplotlib.pyplot as plt

from torchvision import transforms

ModuleNotFoundError: No module named 'dataset'

# Generate Population

In [2]:
password = 'brad1234'
min_len = 2
max_len = 10
# 엄마도 기억력의 한계가 있어서 비밀번호의 최대, 최소 자릿수를 지정해놓았다

def generate_word(length):
    result = ''
    x = ''.join(random.sample(string.ascii_letters + string.digits, k=length))
    return x

# 옛날 옛적 빵형이 지구에 빛이 있으라하여 첫번째 세대가 태어났다
def generate_population(size, min_len, max_len):
    population = []
    for i in range(size):
        # generate words with balanced length
        length = i % (max_len - min_len + 1) + min_len
        population.append(generate_word(length))
    return population

print(generate_word(length=10))
pop = generate_population(size=100, min_len=min_len, max_len=max_len)
pop

sJ6pQyaT9Y


['eZ',
 'Sjq',
 'eYBV',
 'SaBKW',
 '3bswTS',
 'xNlGBte',
 'rt6ZfSGu',
 'MWDurQzY5',
 'nZQLl0T5A7',
 'cj',
 '6eG',
 'eUPS',
 'dh1mp',
 'pMuNvf',
 '8BwyfdT',
 'VzLZ4c1j',
 'zwK0u23TX',
 '0pxcPUytTD',
 'AD',
 'dGm',
 'I5tS',
 'Sy1P8',
 'okOJit',
 '1VKzs6L',
 'nAUaS1lB',
 'aBNg8MQGd',
 'pkV8WrPlej',
 'Lw',
 'qBW',
 '7Miq',
 'JFxY6',
 'aW0Eim',
 'rqaW9fB',
 'qspgi3Rb',
 'VrFb6ynYE',
 'KLtqfYQMSZ',
 '3N',
 'mIW',
 'Yz1d',
 'pT0KS',
 'Vx9mGe',
 'EBUjLq9',
 'jFl8LZbw',
 'jSugNmV9n',
 'mVcBorQlf7',
 'dx',
 'ORM',
 'ecDw',
 'VyPfS',
 'BcHOuC',
 'tLv8Vpr',
 'CuRXU8aK',
 'E6YNqdnk4',
 'Ye918kuQRg',
 'd1',
 'eUQ',
 'uFK4',
 'R1wWz',
 'D3Taw8',
 'CHKOWqY',
 'wBWDpeqT',
 'Rq9w24Kum',
 'tpHuEaz4PO',
 'PF',
 '5NC',
 'gTcm',
 '1UoSB',
 'XdpOxe',
 'i31XnvQ',
 't0b6iWjx',
 '1X2eaYsnB',
 'MrUsG95N0F',
 'Tt',
 'OjN',
 'zlce',
 '6pDRd',
 'KzQBEb',
 'yOAzrbf',
 '56Yd2cDt',
 'gxX2Hhitk',
 'hng6M8qB4G',
 'Op',
 'acH',
 'iXxE',
 'i95Ro',
 'jGwSga',
 'XM905zL',
 'u8tsz23O',
 'meAL6uCkd',
 'GnmVJ0oq1Z',
 '3s',
 'H

# Fitness Function

In [11]:
def fitness(password, test_word):
    score = 0

    if len(password) != len(test_word):
        return score
    
    # if fit length, I'll give you score 0.5
    len_score = 0.5
    score += len_score

    for i in range(len(password)):
        if password[i] == test_word[i]:
            score += 1

    # 백점 만점에 몇점?
    return score / (len(password) + len_score) * 100

fitness('abcdE', 'abcde')

81.81818181818183

# Compute Performance and Select Survivors

In [4]:
def compute_performace(population, password):
    performance_list = []
    for individual in population:
        score = fitness(password, individual)
        
        # 일단 길이는 맞았다..
        # we can predict length of password
        if score > 0:
            pred_len = len(individual)
        performance_list.append([individual, score])

    population_sorted = sorted(performance_list, key=lambda x: x[1], reverse=True) #x[1]이 점수. 낮은 순으로 정렬
    return population_sorted, pred_len

#성능이 제일 좋은거 elitlism 으로. 살리고, 운이 좋은 애들도 랜덤으로 살림.

def select_survivors(population_sorted, best_sample, lucky_few, password_len):
    next_generation = []

    for i in range(best_sample):
        if population_sorted[i][1] > 0:
            next_generation.append(population_sorted[i][0])

    lucky_survivors = random.sample(population_sorted, k=lucky_few)
    for l in lucky_survivors:
        next_generation.append(l[0])
    
    # generate new population if next_generation is too small
    while len(next_generation) < best_sample + lucky_few:
        next_generation.append(generate_word(length=password_len))
    
    #그냥 섞고 리턴해줌.
    
    random.shuffle(next_generation)
    return next_generation

pop_sorted, pred_len = compute_performace(pop, password)
# 베스트 20, 럭키 20 생성
survivors = select_survivors(pop_sorted, best_sample=20, lucky_few=20, password_len=pred_len) 

print('Password length must be %s' % pred_len)
survivors

Password length must be 8


['dx',
 '6eG',
 'wXrNlJth',
 'w4ueCkb8',
 'rt6ZfSGu',
 'ORM',
 'VzLZ4c1j',
 'ZGxpzrAt',
 'Ye918kuQRg',
 'tsncg',
 'eUQ',
 'nAUaS1lB',
 'E6YNqdnk4',
 'Op',
 't0b6iWjx',
 'CuRXU8aK',
 'uzLwj2Di',
 'hpa1uQMN',
 'acH',
 '1j8Hwua9',
 'hng6M8qB4G',
 'qspgi3Rb',
 'eUPS',
 'u8tsz23O',
 'aW0Eim',
 '7xKO1f4L',
 'mIW',
 '1UoSB',
 '56Yd2cDt',
 '3zFxdm5D',
 'jFl8LZbw',
 'rt6ZfSGu',
 'RVJwOKdu',
 'okOJit',
 'MrUsG95N0F',
 'Niedx2Ut',
 'qspgi3Rb',
 'GnmVJ0oq1Z',
 'Rq9w24Kum',
 'wBWDpeqT']

# Create Children

In [5]:
def create_child(individual1, individual2):
    child = ''
    min_len_ind = min(len(individual1), len(individual2))
    for i in range(min_len_ind):
        if (int(100 * random.random()) < 50):
            child += individual1[i]
        else:
            child += individual2[i]
    return child

# 우리는 그렇게 자비롭지 않기때문에 부모를 내 맘대로 짝지어줄 것이다
def create_children(parents, n_child):
    next_population = []
    #부모가 100이면 5개 생김
    
    for i in range(int(len(parents)/2)):
        for j in range(n_child):
            next_population.append(create_child(parents[i], parents[len(parents) - 1 - i]))
    return next_population

# 40개 넣어서 살아남는게 각각 5개. 40명 부모 = 20커플. 5명씩 낳아서 100명이 생김.
children = create_children(survivors, 5)

children

['wB',
 'wx',
 'dB',
 'dB',
 'wx',
 'RqG',
 'RqG',
 '6qG',
 '6q9',
 'RqG',
 'GXmVl0tq',
 'GXmNJ0oq',
 'wnmNlJoh',
 'wnmVl0tq',
 'GXrNJJoq',
 'qsugikR8',
 'w4peC3Rb',
 'qsuei3Rb',
 'q4ueCkRb',
 'w4ueC3R8',
 'NieZx2Gt',
 'Ni6df2Ut',
 'Ni6dxSGt',
 'rtedx2Gu',
 'NieZxSGt',
 'MRU',
 'ORU',
 'ORM',
 'ORU',
 'MrU',
 'VkLZ4t',
 'VkLJit',
 'ozLZ4t',
 'okLJ4t',
 'okLZit',
 'ZVJwOrAu',
 'RVJpOrdu',
 'RGJpzrAt',
 'ZGJpOrAu',
 'ZGxwzKdu',
 're9ZfkuQ',
 'rt9Z8SGu',
 're9ZfkGQ',
 'Yt918kGQ',
 'Ye618kGu',
 'jsn8g',
 'tsncL',
 'tFlcL',
 'jFl8L',
 'tsn8g',
 '3UF',
 '3UF',
 '3zQ',
 '3zF',
 '3UQ',
 '5AUaS1lt',
 '56Ya2cDt',
 'nAUaSclB',
 'nAUa21lB',
 '5AYa2clt',
 'E6oNq',
 '1UYNB',
 'EUYNq',
 'E6YSq',
 '16YNB',
 'Op',
 'mp',
 'Op',
 'Op',
 'Op',
 '7xKO1Wjx',
 't0KO1W4L',
 't0KOif4L',
 't0bO1fjx',
 'txK61f4L',
 'Cu0XUm',
 'aW0Ei8',
 'aWREU8',
 'aW0EUm',
 'Cu0Xim',
 'u8Lwj2DO',
 'u8Lsj2Di',
 'uzLwz23O',
 'u8tsj2Di',
 'u8twj2Di',
 'eUP1',
 'epPS',
 'hpaS',
 'hUa1',
 'hUa1',
 'qsp',
 'acH',
 'qsp',
 'acH',
 'q

# Mutation

In [6]:
def mutate_word(word):
    idx = int(random.random() * len(word))
    if (idx == 0):
        word = random.choice(string.ascii_letters + string.digits) + word[1:]
    else:
        word = word[:idx] + random.choice(string.ascii_letters + string.digits) + word[idx+1:]
    return word

def mutate_population(population, chance_of_mutation):
    for i in range(len(population)):
        if random.random() * 100 < chance_of_mutation:
            population[i] = mutate_word(population[i])
    return population

# chance = 10ㅆ로 확률로 변이를 시키겠따.
new_generation = mutate_population(population=children, chance_of_mutation=10)

new_generation

['wB',
 'wx',
 'dB',
 'dB',
 'wx',
 'RqG',
 'RqD',
 '6qG',
 '6q9',
 'RqG',
 'GXmVl0tq',
 'GXmNJ0oq',
 'wnmNlJoh',
 'wnmVl0tq',
 'GXrNJJoq',
 'qmugikR8',
 'w4peC3Rb',
 'qsuei3Rb',
 'q4ueCkRb',
 'w4ueC3R8',
 'NieZx2Gt',
 'Ni6df2Ut',
 'Ni6dxSGt',
 'rtedx2Gu',
 'NieZxSGt',
 'MiU',
 'ORU',
 'ORM',
 'ORU',
 'MrU',
 'VkLZ4t',
 'VkLJit',
 'ozLZ4t',
 'okLJ4t',
 'okLZit',
 'ZVJwOrAu',
 'RVJpOrdu',
 'RGJpzrAt',
 'ZGJpOrAu',
 'ZGxwzKdu',
 're9ZfkuQ',
 'rt9Z8SGu',
 're9ZfkGQ',
 'Yt918kGQ',
 'Ye618kGu',
 'jsn8g',
 'tsncL',
 'tFlcL',
 'jFl8L',
 'tsn8g',
 '3UF',
 '3UF',
 '3zQ',
 '3zF',
 '3UQ',
 '5AUaS1lt',
 '56Ya2cDt',
 'nAUaSclB',
 'nAUa21lB',
 '5AYa2clt',
 'E6oNq',
 '1UYNB',
 'EUYNq',
 'E6YSq',
 '16YNB',
 'Op',
 '9p',
 'Op',
 'Op',
 'Op',
 '7xKO1Wjx',
 't0KO1W4L',
 't0KOif4L',
 't0bO1fjx',
 'txK61f4L',
 'Cu0XUm',
 'aW0Si8',
 'aWREU8',
 'aW0EUm',
 'Cu0Xim',
 'u8Lwj2DO',
 'u8Lsj2Di',
 'uzLwz23O',
 'u8tsj2Di',
 'u8twj2Di',
 'eUP1',
 'epPS',
 'hpaS',
 'hUa1',
 'hUa1',
 'qsp',
 'acH',
 'qsw',
 'acH',
 'q

# Test

In [7]:
password = 'bBanGhyONg'
n_generation = 300
population = 100
best_sample = 20
lucky_few = 20
n_child = 5
chance_of_mutation = 10

pop = generate_population(size=population, min_len=min_len, max_len=max_len)

for g in range(n_generation):
    pop_sorted, pred_len = compute_performace(population=pop, password=password)

    if int(pop_sorted[0][1]) == 100:
        print('SUCCESS! The password is %s' % (pop_sorted[0][0]))
        break
    
    survivors = select_survivors(population_sorted=pop_sorted, best_sample=best_sample, lucky_few=lucky_few, password_len=pred_len)
    
    children = create_children(parents=survivors, n_child=n_child)

    new_generation = mutate_population(population=children, chance_of_mutation=10)
    
    pop = new_generation
    
    print('===== %sth Generation =====' % (g + 1))
    print(pop_sorted[0])

===== 1th Generation =====
['Qea3du9mxN', 14.285714285714285]
===== 2th Generation =====
['YmcamRIXWf', 4.761904761904762]
===== 3th Generation =====
['4upamcIBWf', 4.761904761904762]
===== 4th Generation =====
['A7D6DaCuYC', 4.761904761904762]
===== 5th Generation =====
['0eylOEFBWf', 4.761904761904762]
===== 6th Generation =====
['MBylOEFxWu', 14.285714285714285]
===== 7th Generation =====
['MBy9D0FxEu', 14.285714285714285]
===== 8th Generation =====
['MBylDEFzWu', 14.285714285714285]
===== 9th Generation =====
['MBcTNRYBWf', 14.285714285714285]
===== 10th Generation =====
['MBy9O0YBEu', 14.285714285714285]
===== 11th Generation =====
['MBclD0FBWC', 14.285714285714285]
===== 12th Generation =====
['MBclDRFBEu', 14.285714285714285]
===== 13th Generation =====
['MBcTNRlBWC', 14.285714285714285]
===== 14th Generation =====
['MBylNRlBWu', 14.285714285714285]
===== 15th Generation =====
['MBclNRbBEu', 14.285714285714285]
===== 16th Generation =====
['MBclDEbBWu', 14.285714285714285]
=====